In [1]:
# import lib
import shapefile
import rasterio
import cv2
from scipy import spatial
from scipy import ndimage as ndi
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

from kneed import KneeLocator
import matplotlib.pyplot as plt
from skimage import io,filters,util,color,img_as_ubyte
from skimage.transform import hough_line, hough_line_peaks
from skimage.morphology import disk, dilation, erosion, remove_small_objects
from skimage.measure import regionprops
from skimage.feature import greycomatrix,greycoprops
import math
import numpy as np
import os
import pandas as pd
import re
from datetime import datetime,date
from tqdm import tqdm

In [2]:
def coor_sort_new(list_p):
    for i in range(len(list_p)):
        j=i
        for j in range(len(list_p)):
            if (list_p[i][0]<list_p[j][0]):
                list_p[i],list_p[j]=list_p[j],list_p[i]
            if (list_p[i][1]>list_p[j][1]):
                list_p[i],list_p[j]=list_p[j],list_p[i]
    if (list_p[0][1]>list_p[1][1]):
        left_top = list_p[0]
        right_top = list_p[1]
        left_bottom = list_p[2]
        right_bottom = list_p[3]
        
    else:
        left_top = list_p[1]
        right_top = list_p[0]
        left_bottom = list_p[3]
        right_bottom = list_p[2]
    sort_p = np.array([left_top,left_bottom,right_top,right_bottom], dtype="float64")
    return sort_p

In [3]:
# The canopy height of the plot was extracted in batches
def plot_height(chm_dir, RTK_information_path, mask_path, scale_factor):
    mask_line = io.imread(mask_path)
    selem = disk(7)
    Grid_Image = dilation(mask_line, selem)
    Grid_Segment = np.logical_not(Grid_Image) 
    Grid_Segment_Refine = erosion(Grid_Segment, disk(3))
    list_bbox_area = []
    Labelled_Plot_Img_o, num_features_o = ndi.measurements.label(Grid_Segment_Refine)
    for region in regionprops(Labelled_Plot_Img_o):
        list_bbox_area.append(region.area)
    re_area = np.max(list_bbox_area)*0.4
    Grid_Segment_Refine = remove_small_objects(Grid_Segment_Refine, re_area)
    Labelled_Plot_Img, num_features = ndi.measurements.label(Grid_Segment_Refine)
    #print(num_features)
    test_dir=os.path.abspath(os.path.dirname(chm_dir))
    try:
        os.mkdir(test_dir+'\\'+'process')
        pro_dir=test_dir+'\\'+'process'

        os.mkdir(test_dir+'\\'+'mask')
        mask_dir=test_dir+'\\'+'mask'

        os.mkdir(test_dir+'\\'+'result')
        result_dir = test_dir+'\\'+'result'
    except FileExistsError:
        pro_dir=test_dir+'\\'+'process'
        mask_dir=test_dir+'\\'+'mask'
        result_dir = test_dir+'/'+'result'
        
    sf = shapefile.Reader(RTK_information_path,'rb')
    shapes = sf.shapes()
    list_p = []
    num=0
    for i in shapes:
        list_p.append(shapes[num].points[0])
        num+=1
        
    geo_coords = coor_sort_new(list_p)
    
    try:
        os.mkdir(result_dir+'\\'+'height')
        height_dir=result_dir+'\\'+'height'

    except FileExistsError:
        height_dir=result_dir+'\\'+'height'
    
    chm_path = os.listdir(chm_dir)
    for name in tqdm(range(len(chm_path))):
        
        dataset = rasterio.open(chm_dir+'\\'+chm_path[name])
        upper_left_x, upper_left_y = (geo_coords[0][0],geo_coords[0][1])
        upper_left_row, upper_left_col = dataset.index(upper_left_x, upper_left_y)
        lower_left_x, lower_left_y = (geo_coords[1][0],geo_coords[1][1])
        lower_left_row, lower_left_col = dataset.index(lower_left_x, lower_left_y)
        upper_right_x, upper_right_y = (geo_coords[2][0],geo_coords[2][1])
        upper_right_row, upper_right_col = dataset.index(upper_right_x, upper_right_y)
        lower_right_x, lower_right_y = (geo_coords[3][0],geo_coords[3][1])
        lower_right_row, lower_right_col = dataset.index(lower_right_x, lower_right_y)

        columns_1 = np.int64(math.sqrt((upper_left_row-lower_left_row)**2
                                      + (upper_left_col-lower_left_col)**2))
        columns_2 = np.int64(math.sqrt((upper_right_row-lower_right_row)**2
                                      + (upper_right_col-lower_right_col)**2))
        columns_val = np.max((columns_1,columns_2))

        rows_1 = np.int64(math.sqrt((upper_left_row-upper_right_row)**2
                                   + (upper_left_col-upper_right_col)**2))
        rows_2 = np.int64(math.sqrt((lower_left_row-lower_right_row)**2
                                   + (lower_left_col-lower_right_col)**2))
        rows_val = np.max((rows_1,rows_2))

        img_tif = io.imread(chm_dir+'\\'+chm_path[name])
        pts1 = np.float32([[upper_left_col,upper_left_row],[upper_right_col,upper_right_row],[lower_left_col,lower_left_row],
                           [lower_right_col,lower_right_row]])
        pts2 = np.float32([[0,0],[rows_val,0],[0,columns_val],[rows_val,columns_val]])
        M_PT = cv2.getPerspectiveTransform(pts1,pts2)
        dst_PT = cv2.warpPerspective(img_tif,M_PT,(rows_val,columns_val))

        # Extract the height
        list_cencoor = list()
        Centroid_coordinates = list()
        height_list_max = list()
        height_list_mean = list()
        height_list_median = list()
        height_list_1 = list()
        height_list_3 = list()
        height_list_5 = list()
        height_list_10 = list()
        canopy_list= list()
        
        for region in regionprops(Labelled_Plot_Img):
            minr, minc, maxr, maxc = region.bbox
            
            col_val = ((maxc-minc)/2)*scale_factor  
            row_val = ((maxr-minr)/2)*scale_factor  
            
            list_cencoor.append(region.centroid[1])
            Centroid_coordinates.append(region.centroid)
            roi=dst_PT[int(region.centroid[0]-row_val):int(region.centroid[0]+row_val),
                       int(region.centroid[1]-col_val):int(region.centroid[1]+col_val)]
            roi = roi[roi>=0]
            roi = roi[roi<=3]
            height_list_max.append(np.max(roi)*100)
            height_list_mean.append(np.mean(roi)*100)
            height_list_median.append(np.median(roi)*100)
            height_list_1.append(np.mean(roi[roi>np.percentile(roi, 99)])*100)
            height_list_3.append(np.mean(roi[roi>np.percentile(roi, 97)])*100)
            height_list_5.append(np.mean(roi[roi>np.percentile(roi, 95)])*100)
            height_list_10.append(np.mean(roi[roi>np.percentile(roi, 90)])*100)
            
        #rows = np.unique(np.array(Centroid_coordinates)[:,0]).shape[0]
        diff_rows = np.diff(np.array(Centroid_coordinates)[:,0])
        thresh = filters.threshold_isodata(diff_rows)
        #print(thresh,diff_rows)
        rows_fil=list(filter(lambda x:x>thresh, diff_rows))
        rows_fil = len(rows_fil)+1
        columns = int(len(Centroid_coordinates)/rows_fil)
        row_index=[]
        column_index=[]
        for i in range(int(rows_fil)):
            for j in range(int(columns)):
                row_index.append(i+1)
                column_index.append(j+1)
        
        Centroid_coordinates_y_array = np.array(list_cencoor)
        height_max_array = np.array(height_list_max)
        height_mean_array = np.array(height_list_mean)
        height_median_array = np.array(height_list_median)
        height_1_array = np.array(height_list_1)
        height_3_array = np.array(height_list_3)
        height_5_array = np.array(height_list_5)
        height_10_array = np.array(height_list_10)        
        height_max_new = []
        height_mean_new = []
        height_median_new= []
        height_1_new = []
        height_3_new = []
        height_5_new = []
        height_10_new = []
        for n in range(0, rows_fil):
            sub_array_y = Centroid_coordinates_y_array[n*columns:columns+n*columns]
            sub_array_max_heights = height_max_array[n*columns:columns+n*columns]
            sub_array_mean_heights = height_mean_array[n*columns:columns+n*columns]
            sub_array_median_heights = height_median_array[n*columns:columns+n*columns]
            sub_array_1_height = height_1_array[n*columns:columns+n*columns]
            sub_array_3_height = height_3_array[n*columns:columns+n*columns]
            sub_array_5_height = height_5_array[n*columns:columns+n*columns]
            sub_array_10_height = height_10_array[n*columns:columns+n*columns]
            rank = np.argsort(sub_array_y)
            sub_array_max_heights = sub_array_max_heights[rank]
            sub_array_mean_heights = sub_array_mean_heights[rank]
            sub_array_median_heights = sub_array_median_heights[rank]
            sub_array_1_height = sub_array_1_height[rank]
            sub_array_3_height = sub_array_3_height[rank]
            sub_array_5_height = sub_array_5_height[rank]
            sub_array_10_height = sub_array_10_height[rank]
            for i in range(0,sub_array_max_heights.shape[0]):
                sub_element_max = sub_array_max_heights[i]
                sub_element_mean = sub_array_mean_heights[i]
                sub_element_median = sub_array_median_heights[i]
                sub_element_1 = sub_array_1_height[i]
                sub_element_3 = sub_array_3_height[i]
                sub_element_5 = sub_array_5_height[i]
                sub_element_10 = sub_array_10_height[i]

                height_max_new.append(sub_element_max)
                height_mean_new.append(sub_element_mean)
                height_median_new.append(sub_element_median)
                height_1_new.append(sub_element_1)
                height_3_new.append(sub_element_3)
                height_5_new.append(sub_element_5)
                height_10_new.append(sub_element_10)

        dt = pd.DataFrame({'rows':row_index, 'columns':column_index,'max':height_max_new,'mean':height_mean_new,'median':height_median_new,
                   '1%':height_1_new,'3%':height_3_new,'5%':height_5_new,'10%':height_10_new})
        dt.to_csv(height_dir+'\\'+chm_path[name][3:-4]+'_canopy_height'+'.csv',encoding="gbk")    
        display(dt)

In [4]:
# Merge the CSV file of canopy height
def merge_height(canopy_height_dir, cho_label):
    height_path = os.listdir(canopy_height_dir)
    test_dir=os.path.abspath(os.path.dirname(canopy_height_dir))
    
    try:
        os.mkdir(test_dir+'\\'+'height_merge')
        merge_dir=test_dir+'\\'+'height_merge'
    except FileExistsError:
        merge_dir=test_dir+'\\'+'height_merge'
        
    df_ext = pd.read_csv(canopy_height_dir+'\\'+height_path[0])
    dt_merge = pd.DataFrame({'rows':df_ext['rows'].values, 'columns':df_ext['columns'].values})
    for name in range(len(height_path)):
        name_str = height_path[name]
        name_num=re.findall(r"\d+\.?\d*",name_str)
        num_max=max(name_num, key = len)
        year, mon, day = num_max[0:4],num_max[4:6],num_max[6:]
        dt_date = date(int(year), int(mon), int(day))
        df_height = pd.read_csv(canopy_height_dir+'\\'+height_path[name])
        plot_he = df_height[cho_label]
        dt_merge[dt_date]=plot_he.values
    
    merge_path = merge_dir+'\\'+'canopy_height_merge.csv'
    dt_merge.to_csv(merge_path,encoding="gbk")
    display(dt_merge)
    return(merge_path)

In [5]:
# dynamic traits
def height_max_rate(merge_height_path, date_start, batch_number):
    test_dir=os.path.abspath(os.path.dirname(merge_height_path))
    test_dir=os.path.abspath(os.path.dirname(test_dir))
    try:
        os.mkdir(test_dir+'\\'+'height_max_rate')
        height_max_rate_dir=test_dir+'\\'+'height_max_rate'
    except FileExistsError:
        height_max_rate_dir=test_dir+'\\'+'height_max_rate'
        
    df_height = pd.read_csv(merge_height_path)
    date_df = df_height.columns[3:]
    list_days = []
    for i in date_df:
        try:
            year_i, mon_i, day_i = i.split('-')
        except ValueError:
            year_i, mon_i, day_i = i.split('/')
        days_num = (datetime(int(year_i), int(mon_i), int(day_i)) - date_start).days
        list_days.append(np.int(days_num))
    list_data = []
    list_entry = []
    for data in df_height.itertuples():
        list_data.append(data[4:])
        list_entry.append(data[1])
    date_ = pd.date_range(date_start, date_df[-1])
    date_all = date_.date
    
    list_fit = []
    list_data_max = []
    list_date_max = []

    height_growth_rate = []
    height_growth_days = []
    for i in range(len(list_data)):
        entry_no = list_entry[i]
        x = np.asarray(list_days)
        y = np.asarray(list_data[i])
        x_real = np.asarray(list_days[0:])
        y_real = np.asarray(list_data[i][0:])

        list_x_cho = []
        list_y_cho = []
        groups = int(np.round(y_real.shape[0] / batch_number))
        #print(y_real.shape[0])
        step = int(y_real.shape[0] / groups)+1
        for j in range(0, y_real.shape[0], step):
            y_div = y_real[j : j + step]
            x_div = x_real[j : j + step]
            percentile = np.percentile(y_div, (25, 50, 75), interpolation='midpoint')
            Q1 = percentile[0]
            Q3 = percentile[2]

            y_cho = y_div[y_div <= Q3]
            y_cho_fin = y_cho[y_cho >= Q1]

            x_cho = x_div[y_div <= Q3]
            x_cho_fin = x_cho[y_cho >= Q1]

            list_x_cho.extend(x_cho_fin)
            list_y_cho.extend(y_cho_fin)

        x_fin = np.asarray(list_x_cho)
        y_fin = np.asarray(list_y_cho)
        x_all = np.arange(0, list_days[-1]+1)

        def gaus(x,a,x0,sigma):
            return a * np.exp(-(x-x0)**2/(2*sigma**2))
        peak_value = y_fin.max()
        mean = x_fin[y_fin.argmax()]
        sigma = mean - np.where(y_fin > peak_value * np.exp(-.5))[0][0]
        try:
            popt,pcov = curve_fit(gaus, x_fin, y_fin, p0=[peak_value, mean, sigma])

            list_fit.append(gaus(x_all,*popt))
            list_poly = []
            for l in x_all:
                list_poly.append(gaus(l,*popt))

            list_date_max.append(date_all[list_poly.index(max(list_poly))])
            list_data_max.append(max(list_poly))

            kneedle_cov_inc = KneeLocator(x_all,
                      gaus(x_all,*popt),
                      curve='convex',
                      direction='increasing',
                      online=True)
            fast_date_st = date_all[kneedle_cov_inc.knee]
            fast_height_st = gaus(kneedle_cov_inc.knee,*popt)

            kneedle_con_inc = KneeLocator(x_all,
                              gaus(x_all,*popt),
                              curve='concave',
                              direction='increasing',
                              online=True)
            if date_all[kneedle_cov_inc.knee] < date_all[kneedle_con_inc.knee]:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
            elif date_all[kneedle_cov_inc.knee] < date_all[list_poly.index(max(list_poly))]:
                fast_date_en = date_all[list_poly.index(max(list_poly))]
                fast_height_en = max(list_poly)
            else:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
                try:
                    year_0, mon_0, day_0 = date_df[0].split('-')
                except ValueError:
                    year_0, mon_0, day_0 = date_df[0].split('/')
                fast_date_st = date(int(year_0), int(mon_0), int(day_0))
                fast_height_st = y_real[0]

            num_day = fast_date_en - fast_date_st
            height_dif = fast_height_en - fast_height_st
            growth_rate = height_dif/num_day.days
            height_growth_days.append(num_day.days)
            height_growth_rate.append(growth_rate)
            
        except RuntimeError:
            popt,pcov = curve_fit(gaus, x_fin, y_fin, p0=[peak_value, mean, sigma],maxfev=400000)

            list_fit.append(gaus(x_all,*popt))
            list_poly = []
            for l in x_all:
                list_poly.append(gaus(l,*popt))

            list_date_max.append(date_all[list_poly.index(max(list_poly))])
            list_data_max.append(max(list_poly))

            kneedle_cov_inc = KneeLocator(x_all,
                      gaus(x_all,*popt),
                      curve='convex',
                      direction='increasing',
                      online=True)
            fast_date_st = date_all[kneedle_cov_inc.knee]
            fast_height_st = gaus(kneedle_cov_inc.knee,*popt)

            kneedle_con_inc = KneeLocator(x_all,
                              gaus(x_all,*popt),
                              curve='concave',
                              direction='increasing',
                              online=True)
            if date_all[kneedle_cov_inc.knee] < date_all[kneedle_con_inc.knee]:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
            elif date_all[kneedle_cov_inc.knee] < date_all[list_poly.index(max(list_poly))]:
                fast_date_en = date_all[list_poly.index(max(list_poly))]
                fast_height_en = max(list_poly)
            else:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
                try:
                    year_0, mon_0, day_0 = date_df[0].split('-')
                except ValueError:
                    year_0, mon_0, day_0 = date_df[0].split('/')
                fast_date_st = date(int(year_0), int(mon_0), int(day_0))
                fast_height_st = y_real[0]

            num_day = fast_date_en - fast_date_st
            height_dif = fast_height_en - fast_height_st
            growth_rate = height_dif/num_day.days
            height_growth_days.append(num_day.days)
            height_growth_rate.append(growth_rate)
            
    dt_height_growth_rate = pd.DataFrame({'rows':df_height['rows'].values, 'columns':df_height['columns'].values,
                                      'max canopy height date':list_date_max, 'max canopy height(cm)':list_data_max,
                                      'fast canopy height growth days':height_growth_days,
                                      'fast canopy height growth rate(cm/day)':height_growth_rate})
    dt_height_growth_rate.to_csv(height_max_rate_dir+'\\'+'canopy_height_max_rate'+'.csv',encoding="gbk")
    display(dt_height_growth_rate)

In [6]:
# Don't denoising, extraction dynamic traits
def non_deno_height_max_rate(merge_height_path, date_start):
    test_dir=os.path.abspath(os.path.dirname(merge_height_path))
    test_dir=os.path.abspath(os.path.dirname(test_dir))
    try:
        os.mkdir(test_dir+'\\'+'height_max_rate')
        height_max_rate_dir=test_dir+'\\'+'height_max_rate'
    except FileExistsError:
        height_max_rate_dir=test_dir+'\\'+'height_max_rate'
        
    df_height = pd.read_csv(merge_height_path)
    date_df = df_height.columns[3:]
    list_days = []
    for i in date_df:
        try:
            year_i, mon_i, day_i = i.split('-')
        except ValueError:
            year_i, mon_i, day_i = i.split('/')
        days_num = (datetime(int(year_i), int(mon_i), int(day_i)) - date_start).days
        list_days.append(np.int(days_num))
    list_data = []
    list_entry = []
    for data in df_height.itertuples():
        list_data.append(data[4:])
        list_entry.append(data[1])
    date_ = pd.date_range(date_start, date_df[-1])
    date_all = date_.date
    
    list_fit = []
    list_data_max = []
    list_date_max = []

    height_growth_rate = []
    height_growth_days = []
    for i in range(len(list_data)):
        entry_no = list_entry[i]
        x = np.asarray(list_days)
        y = np.asarray(list_data[i])
        x_real = np.asarray(list_days[0:])
        y_real = np.asarray(list_data[i][0:])

        list_x_cho = []
        list_y_cho = []
        x_all = np.arange(0, list_days[-1]+1)
        
        def gaus(x,a,x0,sigma):
            return a * np.exp(-(x-x0)**2/(2*sigma**2))
        peak_value = y_real.max()
        mean = x_real[y_real.argmax()]
        sigma = mean - np.where(y_real > peak_value * np.exp(-.5))[0][0]
        try:
            popt,pcov = curve_fit(gaus, x_real, y_real, p0=[peak_value, mean, sigma])

            list_fit.append(gaus(x_all,*popt))
            list_poly = []
            for l in x_all:
                list_poly.append(gaus(l,*popt))

            list_date_max.append(date_all[list_poly.index(max(list_poly))])
            list_data_max.append(max(list_poly))

            kneedle_cov_inc = KneeLocator(x_all,
                      gaus(x_all,*popt),
                      curve='convex',
                      direction='increasing',
                      online=True)
            fast_date_st = date_all[kneedle_cov_inc.knee]
            fast_height_st = gaus(kneedle_cov_inc.knee,*popt)

            kneedle_con_inc = KneeLocator(x_all,
                              gaus(x_all,*popt),
                              curve='concave',
                              direction='increasing',
                              online=True)
            if date_all[kneedle_cov_inc.knee] < date_all[kneedle_con_inc.knee]:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
            elif date_all[kneedle_cov_inc.knee] < date_all[list_poly.index(max(list_poly))]:
                fast_date_en = date_all[list_poly.index(max(list_poly))]
                fast_height_en = max(list_poly)
            else:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
                try:
                    year_0, mon_0, day_0 = date_df[0].split('-')
                except ValueError:
                    year_0, mon_0, day_0 = date_df[0].split('/')
                fast_date_st = date(int(year_0), int(mon_0), int(day_0))
                fast_height_st = y_real[0]

            num_day = fast_date_en - fast_date_st
            height_dif = fast_height_en - fast_height_st
            growth_rate = height_dif/num_day.days
            height_growth_days.append(num_day.days)
            height_growth_rate.append(growth_rate)
            
        except RuntimeError:
            popt,pcov = curve_fit(gaus, x_real, y_real, p0=[peak_value, mean, sigma],maxfev=400000)

            list_fit.append(gaus(x_all,*popt))
            list_poly = []
            for l in x_all:
                list_poly.append(gaus(l,*popt))

            list_date_max.append(date_all[list_poly.index(max(list_poly))])
            list_data_max.append(max(list_poly))

            kneedle_cov_inc = KneeLocator(x_all,
                      gaus(x_all,*popt),
                      curve='convex',
                      direction='increasing',
                      online=True)
            fast_date_st = date_all[kneedle_cov_inc.knee]
            fast_height_st = gaus(kneedle_cov_inc.knee,*popt)

            kneedle_con_inc = KneeLocator(x_all,
                              gaus(x_all,*popt),
                              curve='concave',
                              direction='increasing',
                              online=True)
            if date_all[kneedle_cov_inc.knee] < date_all[kneedle_con_inc.knee]:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
            elif date_all[kneedle_cov_inc.knee] < date_all[list_poly.index(max(list_poly))]:
                fast_date_en = date_all[list_poly.index(max(list_poly))]
                fast_height_en = max(list_poly)
            else:
                fast_date_en = date_all[kneedle_con_inc.knee]
                fast_height_en = gaus(kneedle_con_inc.knee,*popt)
                try:
                    year_0, mon_0, day_0 = date_df[0].split('-')
                except ValueError:
                    year_0, mon_0, day_0 = date_df[0].split('/')
                fast_date_st = date(int(year_0), int(mon_0), int(day_0))
                fast_height_st = y_real[0]

            num_day = fast_date_en - fast_date_st
            height_dif = fast_height_en - fast_height_st
            growth_rate = height_dif/num_day.days
            height_growth_days.append(num_day.days)
            height_growth_rate.append(growth_rate)
            
    dt_height_growth_rate = pd.DataFrame({'rows':df_height['rows'].values, 'columns':df_height['columns'].values,
                                      'max canopy height date':list_date_max, 'max canopy height(cm)':list_data_max,
                                      'fast canopy height growth days':height_growth_days,
                                      'fast canopy height growth rate(cm/day)':height_growth_rate})
    dt_height_growth_rate.to_csv(height_max_rate_dir+'\\'+'non_deno_canopy_height_max_rate'+'.csv',encoding="gbk")
    display(dt_height_growth_rate)

In [7]:
# Step 1: batch extract plot canopy height
chm_dir = r"D:\AirMeasurer_testdata_20songjiang\choose_field_1_201124\chm"
RTK_information_path = r"D:\AirMeasurer_testdata_20songjiang\rtk_new\sj_20200803_r.shp"
mask_path = r"D:\AirMeasurer_testdata_20songjiang\choose_field_1_201124\mask\_mask.png"
scale_factor = 0.9
plot_height(chm_dir, RTK_information_path, mask_path, scale_factor)

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]

,rows,columns,max,mean,median,1%,3%,5%,10%
0,1,1,6.529972,0.137828,0.099102,1.325900,0.773020,0.624716,0.472122
1,1,2,1.853618,0.153485,0.100440,1.185915,0.858050,0.726112,0.561539
2,1,3,1.631179,0.131879,0.087336,0.885337,0.692149,0.601091,0.473103
3,1,4,43.101966,8.052931,3.923840,36.276841,34.058830,32.597789,29.522160
4,1,5,42.538673,15.436858,14.858335,36.995018,35.037366,33.977938,32.382372
...,...,...,...,...,...,...,...,...,...
471,34,10,78.170025,50.780249,54.183102,73.924100,72.097784,70.999604,69.275850
472,34,11,76.199216,24.294476,23.182134,65.202987,59.805375,57.028055,52.454001
473,34,12,88.850689,40.125373,39.376435,77.340382,73.586863,71.526295,68.403679
474,34,13,73.537010,33.663264,30.425569,67.493141,64.278615,62.503135,59.763187


  7%|█████▌                                                                             | 1/15 [00:01<00:17,  1.23s/it]

,rows,columns,max,mean,median,1%,3%,5%,10%
0,1,1,11.941469,0.272136,0.125755,3.543474,2.153491,1.716438,1.241623
1,1,2,2.725813,0.265384,0.158141,1.489687,1.179055,1.056047,0.894956
2,1,3,2.339435,0.248123,0.132383,1.626885,1.304364,1.161546,0.963669
3,1,4,42.845756,20.578393,23.742327,38.976982,37.241611,36.325735,34.903824
4,1,5,46.460751,29.243329,31.237781,43.584117,42.357466,41.657537,40.545648
...,...,...,...,...,...,...,...,...,...
471,34,10,61.789536,36.738646,39.562079,59.364671,58.084971,57.354093,56.108534
472,34,11,58.737010,18.332544,16.958416,48.514870,45.694971,43.951881,40.754977
473,34,12,63.009053,28.818896,30.981785,57.332247,54.587233,53.319401,51.311904
474,34,13,56.112844,20.431291,22.241434,45.499694,42.379934,40.978914,38.930568


 13%|███████████                                                                        | 2/15 [00:02<00:15,  1.21s/it]

...

 87%|███████████████████████████████████████████████████████████████████████           | 13/15 [00:16<00:02,  1.25s/it]

,rows,columns,max,mean,median,1%,3%,5%,10%
0,1,1,104.192853,6.724465,0.622533,88.361502,75.106579,64.769381,46.312949
1,1,2,59.774262,10.254775,7.509813,49.118003,41.328204,38.231021,33.869338
2,1,3,109.479213,19.219810,16.388920,102.957594,86.562610,71.617496,55.364990
3,1,4,108.974004,61.974847,61.727035,96.683180,92.932564,90.627807,84.932953
4,1,5,142.355764,72.011536,74.045503,108.807850,103.484523,100.572944,95.718884
...,...,...,...,...,...,...,...,...,...
471,34,10,121.687317,76.416409,77.331889,108.956695,105.720937,104.175925,101.161361
472,34,11,91.458869,46.825445,40.347615,84.936947,83.169246,82.045376,79.896665
473,34,12,104.952085,66.172171,67.011917,96.634668,92.942184,90.756810,86.889768
474,34,13,122.365189,46.656317,44.056875,99.366784,88.384199,84.254122,78.551513


 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:17<00:01,  1.25s/it]

,rows,columns,max,mean,median,1%,3%,5%,10%
0,1,1,97.729766,5.363247,1.805949,79.416460,56.269121,45.095634,30.649474
1,1,2,59.094036,9.467917,7.429937,47.631231,39.468950,35.343832,29.909161
2,1,3,116.188729,19.181170,15.346242,94.410968,79.358494,67.788905,52.718598
3,1,4,112.201428,62.799716,61.955404,102.594376,97.732121,94.857138,90.029073
4,1,5,111.742580,67.544317,68.758690,103.575230,99.762553,97.123772,92.685688
...,...,...,...,...,...,...,...,...,...
471,34,10,134.256220,46.844468,44.439149,123.679447,111.626637,101.189375,85.344338
472,34,11,75.598001,39.762214,41.628724,67.534751,64.453501,62.551266,58.155143
473,34,12,99.401116,62.643492,63.311327,95.685440,92.495131,90.296131,86.605293
474,34,13,88.171953,46.105990,46.556774,80.881512,76.084536,73.021865,67.212933


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:18<00:00,  1.25s/it]


In [8]:
# Step 2: Merge the CSV file of the canopy height of the cell
canopy_height_dir = r"D:\AirMeasurer_testdata_20songjiang\choose_field_1_201124\result\height"
cho_label = '10%'
merge_path = merge_height(canopy_height_dir, cho_label)

,rows,columns,2020-07-23,2020-07-24,2020-08-03,2020-08-06,2020-08-10,2020-08-22,2020-08-25,2020-08-28,2020-08-31,2020-09-09,2020-09-13,2020-09-19,2020-09-20,2020-09-26,2020-10-13
0,1,1,0.472122,1.241623,1.072996,0.373532,3.891772,3.404766,2.488323,12.239082,4.184223,42.579401,23.485528,21.400750,36.347520,46.312949,30.649474
1,1,2,0.561539,0.894956,0.956023,0.956765,2.977194,4.405281,4.018611,20.392317,8.797880,21.320112,27.255523,29.319045,24.847004,33.869338,29.909161
2,1,3,0.473103,0.963669,0.805226,0.423413,1.095943,49.692088,47.378090,63.129812,54.149425,52.893454,40.633491,39.203626,41.918993,55.364990,52.718598
3,1,4,29.522160,34.903824,55.421823,63.999283,76.812130,104.795098,102.553773,105.286682,121.623087,94.363970,91.649109,82.283503,84.399772,84.932953,90.029073
4,1,5,32.382372,40.545648,57.377160,61.899763,75.950432,127.972257,131.820250,121.522498,130.918467,112.175012,94.296938,82.333440,85.181606,95.718884,92.685688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,34,10,69.275850,56.108534,75.390017,66.801459,85.047346,112.878335,95.864832,113.269746,134.753883,130.727053,120.945132,112.543344,113.890362,101.161361,85.344338
472,34,11,52.454001,40.754977,46.284240,45.680791,49.954128,62.374258,48.040143,44.309083,62.594169,68.886954,85.520226,92.657828,93.360716,79.896665,58.155143
473,34,12,68.403679,51.311904,70.406586,69.369555,78.803593,91.722053,71.820205,73.399544,90.608507,86.712813,92.009610,96.420777,97.144496,86.889768,86.605293
474,34,13,59.763187,38.930568,47.568938,52.547610,56.972438,70.072311,50.676024,48.836038,66.918027,59.095734,74.860704,102.842367,93.799746,78.551513,67.212933


In [9]:
# The third step: denoising, calculating dynamic traits
merge_height_path = r"D:\AirMeasurer_testdata_20songjiang\choose_field_1_201124\result\height_merge\canopy_height_merge.csv"
date_start = pd.Timestamp('2020-06-15')
batch_number = 4
height_max_rate(merge_height_path, date_start, batch_number)

,rows,columns,max canopy height date,max canopy height(cm),fast canopy height growth days,fast canopy height growth rate(cm/day)
0,1,1,2020-10-13,82.479523,41,1.744454
1,1,2,2020-09-29,42.493110,37,1.061857
2,1,3,2020-09-24,58.533888,55,0.882385
3,1,4,2020-09-03,104.457366,53,1.681881
4,1,5,2020-09-11,118.522496,61,1.558213
...,...,...,...,...,...,...
471,34,10,2020-09-10,117.781637,65,1.438343
472,34,11,2020-10-13,98.723635,65,0.787116
473,34,12,2020-10-06,89.861486,32,0.407814
474,34,13,2020-10-13,92.785805,55,0.648563


In [10]:
# The third step: without denoising, calculate dynamic traits
non_deno_height_max_rate(merge_height_path, date_start)

,rows,columns,max canopy height date,max canopy height(cm),fast canopy height growth days,fast canopy height growth rate(cm/day)
0,1,1,2020-09-29,40.457451,41,0.908302
1,1,2,2020-10-02,33.583779,46,0.651324
2,1,3,2020-09-18,54.879192,51,0.927546
3,1,4,2020-09-08,103.184511,59,1.423165
4,1,5,2020-09-05,117.234964,55,1.790032
...,...,...,...,...,...,...
471,34,10,2020-09-09,120.498975,63,1.535648
472,34,11,2020-09-28,77.033071,72,0.744324
473,34,12,2020-09-24,91.424516,35,0.451120
474,34,13,2020-10-10,79.947055,44,0.253509
